In [ ]:
!pip install evaluate rouge_score
!pip install evaluate rouge_score
!pip install bert-score
!pip install bleurt
!pip install tensorflow


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8bb580abc99c7badfc07618ea633b9abfe5c653208c4e05e0b1c2a2de5cdd0f6
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install tensorflow
!pip install git+https://github.com/google-research/bleurt.git


  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-svy3g4qb
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-svy3g4qb
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456766 sha256=bbcd54be592abb89672d120377ff0e8c9369a7346a686f3716e71489a691c6d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-qki0z275/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT


In [ ]:
import json
import evaluate
import csv
import time
from bert_score import score as bert_score
from bleurt import score as bleurt_score  # BLEURT için gerekli kütüphane

# Dosya ve model ayarları
input_file = "/content/duzeltilmis4b.jsonl"  # JSONL dosyan
output_csv = "model_skorlari.csv"  # Skorların kaydedileceği CSV
model_name = "1b"  # Model adı

# ROUGE metriklerini yükle
rouge = evaluate.load("rouge")

# JSONL okuma
references = []
predictions = []

with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        references.append(f"{data['prompt']} {data['reference']}")
        predictions.append(data["generated_response"])

# Hesaplama zamanı
start_time = time.time()

# ROUGE hesapla
rouge_results = rouge.compute(predictions=predictions, references=references)

# BERTScore hesapla (Türkçe)
P_bert, R_bert, F1_bert = bert_score(predictions, references, lang="tr", rescale_with_baseline=True)
bert_f1_avg = F1_bert.mean().item()

# BLEURT hesapla
bleurt_scorer = bleurt_score.BleurtScorer()  # Varsayılan model yüklenir, internet gerekli
bleurt_scores = bleurt_scorer.score(references=references, candidates=predictions)
bleurt_avg = sum(bleurt_scores) / len(bleurt_scores)

elapsed_time = time.time() - start_time

# Sonuçları CSV'ye kaydet
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)

    # Dosya boşsa başlık ekle
    csv_file.seek(0, 2)
    if csv_file.tell() == 0:
        writer.writerow([
            "model", "ROUGE1", "ROUGE2", "ROUGEL",
            "BERTScore_F1", "BLEURT_avg", "süre_saniye"
        ])

    writer.writerow([
        model_name,
        rouge_results["rouge1"],
        rouge_results["rouge2"],
        rouge_results["rougeL"],
        bert_f1_avg,
        bleurt_avg,
        elapsed_time
    ])

print(f"Skorlar hesaplandı ve '{output_csv}' dosyasına kaydedildi.")


Skorlar hesaplandı ve 'model_skorlari.csv' dosyasına kaydedildi.


In [ ]:
extract_model_response()

In [ ]:
rouge_results_detail = rouge.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)